In [1]:
import json
import numpy as np
from skmultilearn.model_selection import iterative_train_test_split

In [2]:
annos = json.load(open('./dataset/modanet_instances_all.json'))

In [3]:
# would use a default dict but need to make sure all image ids are present
data = {}
for im in annos['images']:
    data[im['id']] = np.zeros(len(annos['categories']))

for a in annos['annotations']:
    data[a['image_id']][a['category_id']-1] += 1

X, y = zip(*data.items())
y = np.stack(y)
X = np.array(X).reshape(-1,1)

In [4]:
# only thing i could find online that attempts multi-label splits
X_train, y_train, X_test, y_test = iterative_train_test_split(X,y, 0.2)

In [6]:
print('distribution train: ', y_train.sum(axis=0))
print('distribution test: ', y_test.sum(axis=0))
a = y_train.sum(axis=0)/y_train.sum()
b = y_test.sum(axis=0)/y_test.sum()
# sorry I dont remember the chapter on hypothesis testing
print('distribution similarity: ', np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b)))

distribution train:  [16630. 11440. 11146. 61848. 20100. 11664.  6985. 18527. 28969.  5633.
 10891.  4343.  4076.]
distribution test:  [ 4246.  2917.  2813. 15311.  5108.  2928.  1812.  4646.  7321.  1442.
  2764.  1105.  1040.]
distribution similarity:  0.9999292789547258


In [20]:
X_train = X_train.reshape(-1)
X_test = X_test.reshape(-1)

train_annos = {}
train_annos['categories'] = annos['categories']
train_annos['images'] = [im for im in annos['images'] if im['id'] in X_train]
train_annos['annotations'] = [a for a in annos['annotations'] if a['image_id'] in X_train]

test_annos = {}
test_annos['categories'] = annos['categories']
test_annos['images'] = [im for im in annos['images'] if im['id'] in X_test]
test_annos['annotations'] = [a for a in annos['annotations'] if a['image_id'] in X_test]

In [29]:
with open('./dataset/modanet_instances_train.json', 'w') as f:
    json.dump(train_annos, f, separators=(',', ':'))

with open('./dataset/modanet_instances_val.json', 'w') as f:
    json.dump(test_annos, f, separators=(',', ':'))